In [ ]:
from influx_kwh import InfluxKwhReader, KwhToPrice, format_entity_id
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime
from dateutil.relativedelta import relativedelta
import numpy as np
import pytz
import math

In [ ]:
start_date = datetime(2024, 1, 1,0, 0, 0).replace(tzinfo=pytz.UTC)
end_date = datetime.today().replace(tzinfo=pytz.UTC)

host=''
port=8086
username = ''
password = ''
dbname = ''

In [ ]:
kwh_reader = InfluxKwhReader(host, port, username, password, dbname)
nb_months = (end_date - start_date).days / 30
import_ranges = [(start_date + relativedelta(days=30 * offset), start_date + relativedelta(days=30 * (offset+1))) for offset in range(math.ceil(nb_months))]
df_stack = pd.concat([kwh_reader.query_data(s, e) for (s,e) in tqdm(import_ranges)])

In [ ]:
df_daily_total = df_stack.groupby(df_stack.index.floor('d')).last()
mean_kwh_per_entity = df_daily_total.apply(np.mean).sort_values(ascending=False)
cols_to_display = list(mean_kwh_per_entity.index[1:])

In [ ]:
df_stack.tail()

In [ ]:
daily_consumption = df_stack[cols_to_display].rename(columns=format_entity_id).resample('D').max()
fig = px.area(daily_consumption.rolling(5).mean().fillna(0.0))
fig.show()

In [ ]:
daily_consumption

In [ ]:
daily_consumption[['Air Compressor 1', 'Air Compressor 2']].sum(axis=1)\
    .rolling(7).mean().plot(figsize=(15,5), grid=True)

In [ ]:
KwhToPrice().price(70, 30)

In [ ]:
daily_consumption.mean().apply(lambda x: KwhToPrice().incremental_price(x, 30)).sort_values(ascending=False)

In [ ]:
daily_consumption['2024-06-01':].mean().apply(lambda x: KwhToPrice().incremental_price(x, 30)).sort_values(ascending=False)